In [41]:
# Installing the repository in Google Colab

# # Disable cell output
# %%capture

# # Download the repository from github
# !git clone https://github.com/5TuX/ml-radiomics-classification.git

# # Set working directory
# %cd ml-radiomics-classification

# # Install dependencies with uv
# !curl -LsSf https://astral.sh/uv/install.sh | sh
# import os
# os.environ["PATH"] = f"/root/.cargo/bin:{os.environ['PATH']}"
# !uv sync

In [42]:
# Imports

import polars as pl
from itertools import groupby
from sklearn.model_selection import train_test_split

In [43]:
# Notebook settings

settings = {
    "dataset-path": "data/Radiomics_binWidth-15_ZScore_NETnNCR_T1CE.csv",
}

# Get the data

In [44]:
def load_radiomics_data(data_path):
    # Load data from CSV file
    df = pl.read_csv(data_path)
    # Exclude some columns already
    cols_to_exclude = [
        "Patient_ID",
        "Group_label",
        "binWidth",
        "Normalization",
        "Subregion",
        "Sequence",
    ]
    df = (
        df.select(
            pl.all().exclude(cols_to_exclude).exclude("^diagnostics_.*$"),
        )
        .drop(  # Move Age column to a better position
            "Age"
        )
        .insert_column(2, df.get_column("Age"))
    )
    return df


df = load_radiomics_data(settings["dataset-path"])

print("Look at the first 5 samples of the dataset:")

df.head()

Look at the first 5 samples of the dataset:


Group,Survival_days,Age,Extent_of_Resection,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,original_shape_Sphericity,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,original_shape_VoxelVolume,original_firstorder_10Percentile,original_firstorder_90Percentile,original_firstorder_Energy,original_firstorder_Entropy,original_firstorder_InterquartileRange,original_firstorder_Kurtosis,original_firstorder_Maximum,original_firstorder_MeanAbsoluteDeviation,original_firstorder_Mean,original_firstorder_Median,original_firstorder_Minimum,original_firstorder_Range,original_firstorder_RobustMeanAbsoluteDeviation,original_firstorder_RootMeanSquared,original_firstorder_Skewness,original_firstorder_TotalEnergy,original_firstorder_Uniformity,original_firstorder_Variance,original_glcm_Autocorrelation,…,wavelet-LLL_glrlm_GrayLevelNonUniformity,wavelet-LLL_glrlm_GrayLevelNonUniformityNormalized,wavelet-LLL_glrlm_GrayLevelVariance,wavelet-LLL_glrlm_HighGrayLevelRunEmphasis,wavelet-LLL_glrlm_LongRunEmphasis,wavelet-LLL_glrlm_LongRunHighGrayLevelEmphasis,wavelet-LLL_glrlm_LongRunLowGrayLevelEmphasis,wavelet-LLL_glrlm_LowGrayLevelRunEmphasis,wavelet-LLL_glrlm_RunEntropy,wavelet-LLL_glrlm_RunLengthNonUniformity,wavelet-LLL_glrlm_RunLengthNonUniformityNormalized,wavelet-LLL_glrlm_RunPercentage,wavelet-LLL_glrlm_RunVariance,wavelet-LLL_glrlm_ShortRunEmphasis,wavelet-LLL_glrlm_ShortRunHighGrayLevelEmphasis,wavelet-LLL_glrlm_ShortRunLowGrayLevelEmphasis,wavelet-LLL_glszm_GrayLevelNonUniformity,wavelet-LLL_glszm_GrayLevelNonUniformityNormalized,wavelet-LLL_glszm_GrayLevelVariance,wavelet-LLL_glszm_HighGrayLevelZoneEmphasis,wavelet-LLL_glszm_LargeAreaEmphasis,wavelet-LLL_glszm_LargeAreaHighGrayLevelEmphasis,wavelet-LLL_glszm_LargeAreaLowGrayLevelEmphasis,wavelet-LLL_glszm_LowGrayLevelZoneEmphasis,wavelet-LLL_glszm_SizeZoneNonUniformity,wavelet-LLL_glszm_SizeZoneNonUniformityNormalized,wavelet-LLL_glszm_SmallAreaEmphasis,wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,wavelet-LLL_glszm_ZoneEntropy,wavelet-LLL_glszm_ZonePercentage,wavelet-LLL_glszm_ZoneVariance,wavelet-LLL_ngtdm_Busyness,wavelet-LLL_ngtdm_Coarseness,wavelet-LLL_ngtdm_Complexity,wavelet-LLL_ngtdm_Contrast,wavelet-LLL_ngtdm_Strength
str,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""HGG""","""289""",60.463,"""GTR""",0.731829,0.41979,21.139285,50.356792,50.358713,59.539903,50.990195,63.85922,15226.333333,36.852581,0.333812,8899.775397,0.584499,15443.0,0.891341,2.406696,43805.647206,0.000994,0.76735,3.905945,4.857057,0.476079,1.57019,1.471689,-0.037217,4.894274,0.322622,1.684222,0.845454,43805.647206,0.99987,0.371107,3.999682,…,2837.692308,1.0,0.0,1.0,56.108317,56.108317,56.108317,1.0,3.683174,314.441224,0.108017,0.183753,23.691856,0.28482,0.28482,0.28482,34.0,1.0,0.0,1.0,6.3840e6,6.3840e6,6.3840e6,1.0,5.470588,0.1609,0.379267,0.379267,0.379267,3.133984,0.002202,6.1777e6,0.0,1e6,0.0,0.0,0.0
"""HGG""","""616""",52.263,"""GTR""",0.805201,0.596898,17.919111,30.020387,34.132096,34.525353,31.144823,35.496479,9073.541667,24.172434,0.73062,2879.455924,0.317346,9160.0,0.511066,1.20927,7040.180407,-3.2034e-16,0.263257,10.595436,3.429786,0.233776,0.801736,0.711371,0.013167,3.416619,0.120232,0.876686,2.380337,7040.180407,1.0,0.125799,1.0,…,931.230769,1.0,0.0,1.0,148.738015,148.738015,148.738015,1.0,4.063811,65.188726,0.068125,0.101663,37.847387,0.148596,0.148596,0.148596,9.0,1.0,0.0,1.0,9.1449e6,9.1449e6,9.1449e6,1.0,1.0,0.111111,0.155282,0

In [45]:
print(
    "Look at statistics (Number of values and undefined values in each column, mean, std etc.:)"
)
df.describe()

Look at statistics (Number of values and undefined values in each column, mean, std etc.:)


statistic,Group,Survival_days,Age,Extent_of_Resection,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,original_shape_Sphericity,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,original_shape_VoxelVolume,original_firstorder_10Percentile,original_firstorder_90Percentile,original_firstorder_Energy,original_firstorder_Entropy,original_firstorder_InterquartileRange,original_firstorder_Kurtosis,original_firstorder_Maximum,original_firstorder_MeanAbsoluteDeviation,original_firstorder_Mean,original_firstorder_Median,original_firstorder_Minimum,original_firstorder_Range,original_firstorder_RobustMeanAbsoluteDeviation,original_firstorder_RootMeanSquared,original_firstorder_Skewness,original_firstorder_TotalEnergy,original_firstorder_Uniformity,original_firstorder_Variance,…,wavelet-LLL_glrlm_GrayLevelNonUniformity,wavelet-LLL_glrlm_GrayLevelNonUniformityNormalized,wavelet-LLL_glrlm_GrayLevelVariance,wavelet-LLL_glrlm_HighGrayLevelRunEmphasis,wavelet-LLL_glrlm_LongRunEmphasis,wavelet-LLL_glrlm_LongRunHighGrayLevelEmphasis,wavelet-LLL_glrlm_LongRunLowGrayLevelEmphasis,wavelet-LLL_glrlm_LowGrayLevelRunEmphasis,wavelet-LLL_glrlm_RunEntropy,wavelet-LLL_glrlm_RunLengthNonUniformity,wavelet-LLL_glrlm_RunLengthNonUniformityNormalized,wavelet-LLL_glrlm_RunPercentage,wavelet-LLL_glrlm_RunVariance,wavelet-LLL_glrlm_ShortRunEmphasis,wavelet-LLL_glrlm_ShortRunHighGrayLevelEmphasis,wavelet-LLL_glrlm_ShortRunLowGrayLevelEmphasis,wavelet-LLL_glszm_GrayLevelNonUniformity,wavelet-LLL_glszm_GrayLevelNonUniformityNormalized,wavelet-LLL_glszm_GrayLevelVariance,wavelet-LLL_glszm_HighGrayLevelZoneEmphasis,wavelet-LLL_glszm_LargeAreaEmphasis,wavelet-LLL_glszm_LargeAreaHighGrayLevelEmphasis,wavelet-LLL_glszm_LargeAreaLowGrayLevelEmphasis,wavelet-LLL_glszm_LowGrayLevelZoneEmphasis,wavelet-LLL_glszm_SizeZoneNonUniformity,wavelet-LLL_glszm_SizeZoneNonUniformityNormalized,wavelet-LLL_glszm_SmallAreaEmphasis,wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,wavelet-LLL_glszm_ZoneEntropy,wavelet-LLL_glszm_ZonePercentage,wavelet-LLL_glszm_ZoneVariance,wavelet-LLL_ngtdm_Busyness,wavelet-LLL_ngtdm_Coarseness,wavelet-LLL_ngtdm_Complexity,wavelet-LLL_ngtdm_Contrast,wavelet-LLL_ngtdm_Strength
str,str,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""369""","""236""",236.0,"""129""",368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,…,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0
"""null_count""","""0""","""133""",133.0,"""240""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""mean""",null,null,61.223203,null,0.723042,0.555079,24.485618,45.730192,45.057,51.053797,49.333276,56.808197,21862.7851,32.004908,0.376242,9237.61258,0.923609,22179.336957,1.374584,2.522678,74696.113733,0.007873,0.626598,3.996735,4.162478,0.36554,1.920441,1.882498,0.571002,3.591476,0.261605,1.985013,0.372539,74696.113733,0.996985,0.23679,…,2963.677357,0.994174,0.002913,1.352574,101.552119,169

In [46]:
print("Explore the structure of columns names:")

print(f"{len(df.columns)} columns in total.\n")


def key(colname):
    return colname.split("_")[0]


for column_group, columns in groupby(df.columns, key):
    print(column_group)
    columns = list(columns)
    if len(columns) == 1:
        print(f"\t-> {columns}")
        continue
    columns = [colname[len(column_group) + 1 :] for colname in columns]
    for column_subgroup, subcolumns in groupby(columns, key):
        maxlen, suffix = 3, ""
        subcolumns = [colname[len(column_subgroup) + 1 :] for colname in subcolumns]
        if (lensubcols := len(subcolumns)) > maxlen:
            subcolumns = subcolumns[:maxlen]
            suffix = f"... ({lensubcols} total)"
        print(f"\t{column_subgroup} -> {subcolumns} {suffix}")

Explore the structure of columns names:
1692 columns in total.

Group
	-> ['Group']
Survival
	-> ['Survival_days']
Age
	-> ['Age']
Extent
	-> ['Extent_of_Resection']
original
	shape -> ['Elongation', 'Flatness', 'LeastAxisLength'] ... (14 total)
	firstorder -> ['10Percentile', '90Percentile', 'Energy'] ... (18 total)
	glcm -> ['Autocorrelation', 'ClusterProminence', 'ClusterShade'] ... (24 total)
	gldm -> ['DependenceEntropy', 'DependenceNonUniformity', 'DependenceNonUniformityNormalized'] ... (14 total)
	glrlm -> ['GrayLevelNonUniformity', 'GrayLevelNonUniformityNormalized', 'GrayLevelVariance'] ... (16 total)
	glszm -> ['GrayLevelNonUniformity', 'GrayLevelNonUniformityNormalized', 'GrayLevelVariance'] ... (16 total)
	ngtdm -> ['Busyness', 'Coarseness', 'Complexity'] ... (5 total)
exponential
	firstorder -> ['10Percentile', '90Percentile', 'Energy'] ... (18 total)
	glcm -> ['Autocorrelation', 'ClusterProminence', 'ClusterShade'] ... (24 total)
	gldm -> ['DependenceEntropy', 'Dependenc

In [47]:
print(
    "Keep only Group target variable (drop Survival_days) and keep only original radiomics (drop transformed features)."
)


def filter_radiomics_data(df: pl.DataFrame):
    # Choose columns to keep:

    target_variable = ["Group"]  # Target variable (drop Survival_days)

    # Here we drop the clinical features (too many missing values)
    # clinical_features = ["Age", "Extent_of_Resection"]
    clinical_features = []

    radiomics = [
        col for col in df.columns if col.startswith("original_")
    ]  # Basic radiomic features

    df = df.select(target_variable + clinical_features + radiomics).rename(
        {col: col[9:] for col in radiomics}
    )

    # Drop rows with missing data (only one in this case)
    df = df.drop_nulls()

    return df


df_filtered = filter_radiomics_data(df)

print(
    "Look at statistics after filtering (Number of values and null values in each column, mean, std etc.:)"
)
df_filtered.describe()

Keep only Group target variable (drop Survival_days) and keep only original radiomics (drop transformed features).
Look at statistics after filtering (Number of values and null values in each column, mean, std etc.:)


statistic,Group,shape_Elongation,shape_Flatness,shape_LeastAxisLength,shape_MajorAxisLength,shape_Maximum2DDiameterColumn,shape_Maximum2DDiameterRow,shape_Maximum2DDiameterSlice,shape_Maximum3DDiameter,shape_MeshVolume,shape_MinorAxisLength,shape_Sphericity,shape_SurfaceArea,shape_SurfaceVolumeRatio,shape_VoxelVolume,firstorder_10Percentile,firstorder_90Percentile,firstorder_Energy,firstorder_Entropy,firstorder_InterquartileRange,firstorder_Kurtosis,firstorder_Maximum,firstorder_MeanAbsoluteDeviation,firstorder_Mean,firstorder_Median,firstorder_Minimum,firstorder_Range,firstorder_RobustMeanAbsoluteDeviation,firstorder_RootMeanSquared,firstorder_Skewness,firstorder_TotalEnergy,firstorder_Uniformity,firstorder_Variance,glcm_Autocorrelation,glcm_ClusterProminence,glcm_ClusterShade,…,glrlm_GrayLevelNonUniformity,glrlm_GrayLevelNonUniformityNormalized,glrlm_GrayLevelVariance,glrlm_HighGrayLevelRunEmphasis,glrlm_LongRunEmphasis,glrlm_LongRunHighGrayLevelEmphasis,glrlm_LongRunLowGrayLevelEmphasis,glrlm_LowGrayLevelRunEmphasis,glrlm_RunEntropy,glrlm_RunLengthNonUniformity,glrlm_RunLengthNonUniformityNormalized,glrlm_RunPercentage,glrlm_RunVariance,glrlm_ShortRunEmphasis,glrlm_ShortRunHighGrayLevelEmphasis,glrlm_ShortRunLowGrayLevelEmphasis,glszm_GrayLevelNonUniformity,glszm_GrayLevelNonUniformityNormalized,glszm_GrayLevelVariance,glszm_HighGrayLevelZoneEmphasis,glszm_LargeAreaEmphasis,glszm_LargeAreaHighGrayLevelEmphasis,glszm_LargeAreaLowGrayLevelEmphasis,glszm_LowGrayLevelZoneEmphasis,glszm_SizeZoneNonUniformity,glszm_SizeZoneNonUniformityNormalized,glszm_SmallAreaEmphasis,glszm_SmallAreaHighGrayLevelEmphasis,glszm_SmallAreaLowGrayLevelEmphasis,glszm_ZoneEntropy,glszm_ZonePercentage,glszm_ZoneVariance,ngtdm_Busyness,ngtdm_Coarseness,ngtdm_Complexity,ngtdm_Contrast,ngtdm_Strength
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""368""",368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,…,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,0.723042,0.555079,24.485618,45.730192,45.057,51.053797,49.333276,56.808197,21862.7851,32.004908,0.376242,9237.61258,0.923609,22179.336957,1.374584,2.522678,74696.113733,0.007873,0.626598,3.996735,4.162478,0.36554,1.920441,1.882498,0.571002,3.591476,0.261605,1.985013,0.372539,74696.113733,0.996985,0.23679,1.530893,0.006383,-0.003187,…,2971.81061,0.994031,0.002985,1.526069,100.816023,186.329359,79.43769,0.868483,3.329075,428.802215,0.183117,0.2571,38.653377,0.369848,0.53148,0.32944,34.180723,0.959979,0.02001,1.353266,3.3695e8,5.0550e8,2.9481e8,0.911684,9.541138,0.32483,0.43707,0.604474,0.395219,2.36478,0.008932,1.1143e8,14.646119,826087.014728,0.0014,0.000218,0.055786
"""std""",null,0.155188,0.147826,10.167512,19.301596,16.080009,20.17874,19.738679,21.27765,29860.458179,12.507605,0.155833,8357.631322,0.910963,29878.14703,0.524901,0.51373,101634.496284,0.067677,0.276941,3.48061,1.281748,0.143954,0.490192,0.526195,0.592212,1.55996,0.111509,0.474185,0.726591,101634.496284,0.032312,0.185438,1.140018,0.065065,0.030067,…,2678.237799,0.03868,0.01934,1.1305,143.823868,380.737352,123.451225,0.282625,1.007

In [48]:
print("Split the data in Train and Test subsets.")

training_data, test_data = train_test_split(
    df_filtered,
    test_size=0.20,
    stratify=df_filtered["Group"],
    random_state=42,
)


def check_class_proportions(df: pl.DataFrame):
    summary = (
        df.group_by("Group")
        .agg(pl.len().alias("count"))
        .with_columns((pl.col("count") / pl.col("count").sum()).alias("proportion"))
        .sort("count", descending=True)
    )
    return summary


print("\nCheck class proportions in training dataset:")
print(check_class_proportions(training_data))

print("\nCheck class proportions in test dataset:")
print(check_class_proportions(test_data))

Split the data in Train and Test subsets.

Check class proportions in training dataset:
shape: (2, 3)
┌───────┬───────┬────────────┐
│ Group ┆ count ┆ proportion │
│ ---   ┆ ---   ┆ ---        │
│ str   ┆ u32   ┆ f64        │
╞═══════╪═══════╪════════════╡
│ HGG   ┆ 233   ┆ 0.792517   │
│ LGG   ┆ 61    ┆ 0.207483   │
└───────┴───────┴────────────┘

Check class proportions in test dataset:
shape: (2, 3)
┌───────┬───────┬────────────┐
│ Group ┆ count ┆ proportion │
│ ---   ┆ ---   ┆ ---        │
│ str   ┆ u32   ┆ f64        │
╞═══════╪═══════╪════════════╡
│ HGG   ┆ 59    ┆ 0.797297   │
│ LGG   ┆ 15    ┆ 0.202703   │
└───────┴───────┴────────────┘
